In [1]:
# uncomment and run if mismo is not installed
# %pip install -q git+https://github.com/NickCrews/mismo@main

In [2]:
from __future__ import annotations

import ibis
from ibis import _
from ibis.backends.duckdb import Backend as DuckDBBackend

import mismo

ibis.options.interactive = True

We are going to dedupe the PATSTAT dataset. Every record represents a patent filed,
and our task is to determine which one came from the same inventor.
The end goal is to add a column called `inventor_id` to each patent.
This dataset contains a ground truth label, so we can evaluate how well we did.

`Mismo` contains this as an included dataset so it is easy to get started.
The returned dataset is an Ibis table, which is a lazy representation of a SQL table.
It is similar to a pandas dataframe, but has a few properties that make it much
better for the record linkage use case:

- Since it is SQL backed, it can handle datasets that are larger than memory, in
  the many millions of rows.
- Computation is performed by the powerful SQL backend of your choice: Google BigQuery,
  Apache Spark, Snowflake, etc. For this demo, we use DuckDB, which is a 
  state-of-the-art SQL engine based around a columnar data model
  (ie oriented towards the bulk operations of record linkage)
- Ibis is strongly typed, has a full API, is well-documented, and has good
  integration with the rest of the python data science ecosystem.


In [3]:
conn: DuckDBBackend = ibis.duckdb.connect()
clean = conn.read_parquet("../../data/alaska-addresses-clean.parquet")
dirty = conn.read_parquet("../../data/alaska-addresses-dirty.parquet")
print(clean.count().execute())
clean

315659


┏━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ record_id ┃ state  ┃ city         ┃ postcode ┃ street                 ┃ unit   ┃ lon         ┃ lat       ┃
┡━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ int64     │ string │ string       │ string   │ string                 │ string │ float64     │ float64   │
├───────────┼────────┼──────────────┼──────────┼────────────────────────┼────────┼─────────────┼───────────┤
│         0 │ AK     │ ANCHOR POINT │ 99556    │ 23837 NIKOLAI STREET   │ NULL   │ -151.746357 │ 59.926617 │
│         1 │ AK     │ ANCHOR POINT │ 99556    │ 24338 MISTY LANE       │ NULL   │ -151.739071 │ 59.919594 │
│         2 │ AK     │ ANCHOR POINT │ 99556    │ 24715 STERLING HIGHWAY │ NULL   │ -151.755646 │ 59.913984 │
│         3 │ AK     │ ANCHOR POINT │ 99556    │ 24803 INLET COVE LANE  │ NULL   │ -151.757118 │ 59.912571 │
│         4 │ AK     │ ANCHOR POINT │ 99556    │ 24825 INLET COVE LANE  │ NULL   │ -151.757657 │ 59.912205 │
│         5 │ AK     │ ANCHOR POINT │ 99556    │ 24845 INLET COVE LANE  │ NULL   │ -151.757918 │ 59.911979 │
│         6 │ AK     │ ANCHOR POINT │ 99556    │ 24865 INLET COVE LANE  │ NULL   │ -151.758861 │ 59.911740 │
│         7 │ AK     │ ANCHOR POINT │ 99556    │ 25100 PEPPER ROAD      │ NULL   │ -151.709791 │ 59.908129 │
│         8 │ AK     │ ANCHOR POINT │ 99556    │ 25238 STINKBUG STREET  │ NULL   │ -151.692157 │ 59.906734 │
│         9 │ AK     │ ANCHOR POINT │ 99556    │ 25405 STINKBUG STREET  │ NULL   │ -151.695885 │ 59.903846 │
│         … │ …      │ …            │ …        │ …                      │ …      │           … │         … │
└───────────┴────────┴──────────────┴──────────┴────────────────────────┴────────┴─────────────┴───────────┘

In [4]:
print(dirty.count().execute())
dirty

992329


┏━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ record_id ┃ state  ┃ city     ┃ postcode ┃ street             ┃ unit   ┃
┡━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ int64     │ string │ string   │ string   │ string             │ string │
├───────────┼────────┼──────────┼──────────┼────────────────────┼────────┤
│         0 │ AK     │ ACHORAGE │ 99417    │ 2010 WLEY HARI AVE │ NULL   │
│         1 │ AK     │ ADAK     │ 99546    │ 241 AMCHITKA CIR   │ NULL   │
│         2 │ AK     │ ADAK     │ 99546    │ 24317 SEWARD HWY   │ NULL   │
│         3 │ AK     │ ADAK     │ 99546    │ 351 AURIGA LN      │ NULL   │
│         4 │ AK     │ AKIACHAK │ 99551    │ 12410 HOPA CIR     │ NULL   │
│         5 │ AK     │ AKIACHAK │ 99551    │ 12812 OOMIPK CIR   │ NULL   │
│         6 │ AK     │ AKIACHAK │ 99551    │ 1760 A HEIRLAM DR  │ NULL   │
│         7 │ AK     │ AKIACHAK │ 99551    │ 1985 BRANDILYN ST  │ NULL   │
│         8 │ AK     │ AKIACHAK │ 99551    │ 2503 LN HONDA DR   │ NULL   │
│         9 │ AK     │ AKIACHAK │ 99551    │ 8731 SULTANA DR    │ NULL   │
│         … │ …      │ …        │ …        │ …                  │ …      │
└───────────┴────────┴──────────┴──────────┴────────────────────┴────────┘

In [5]:
def norm_string(s: ibis.ir.StringValue) -> ibis.ir.StringValue:
    return mismo.text.norm_whitespace(s).upper().nullif("")


normed_strings = ibis.selectors.across(ibis.selectors.of_type(str), norm_string)

clean = clean.mutate(normed_strings).cache()
dirty = dirty.mutate(normed_strings).cache()

In [6]:
from mismo import KeyLinker

address_linker = KeyLinker("city", "postcode", "street", "unit")
address_linker.key_counts(dirty)

┏━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━┓
┃ city      ┃ postcode ┃ street              ┃ unit      ┃ n     ┃
┡━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━┩
│ string    │ string   │ string              │ string    │ int64 │
├───────────┼──────────┼─────────────────────┼───────────┼───────┤
│ ANCHORAGE │ 99503    │ 520 E 34TH AVE      │ UNIT 102  │  1419 │
│ ANCHORAGE │ 99503    │ 3801 CENTERPOINT DR │ UNIT 200  │   769 │
│ ANCHORAGE │ 99503    │ 101 W BENSON BLVD   │ UNIT 503  │   766 │
│ ANCHORAGE │ 99503    │ 3333 DENALI ST      │ UNIT 200  │   586 │
│ ANCHORAGE │ 99503    │ 2525 C ST           │ UNIT 500  │   562 │
│ JUNEAU    │ 99801    │ 201 MAIN ST         │ UNIT 300  │   503 │
│ ANCHORAGE │ 99503    │ 101 W BENSON BLVD   │ UNIT 101  │   434 │
│ JUNEAU    │ 99801    │ 211 4TH ST          │ UNIT 306  │   430 │
│ ANCHORAGE │ 99503    │ 2550 DENALI ST      │ UNIT 1000 │   409 │
│ JUNEAU    │ 99801    │ 230 S FRANKLIN ST   │ UNIT 602  │   388 │
│ …         │ …        │ …                   │ …         │     … │
└───────────┴──────────┴─────────────────────┴───────────┴───────┘

In [7]:
many_linkages = [
    KeyLinker("city", "street", "unit").__link__(clean, dirty),
    KeyLinker("postcode", "street", "unit").__link__(clean, dirty),
]
linkage = mismo.linkage.intersect(many_linkages)
linkage

In [8]:
linkage.__sublinkages__()

(KeyLinkage(
     keys=(city, street, unit)
     nleft=315_659
     nright=992_329
     nlinks=102
 ),
 KeyLinkage(
     keys=(postcode, street, unit)
     nleft=315_659
     nright=992_329
     nlinks=102
 ))

In [9]:
linkage2 = mismo.linkage.union(
    [linkage, KeyLinker("city", _.street[:5], "unit").__link__(clean, dirty)]
)
linkage2

In [10]:
linkage2.__sublinkages__()

(<mismo.linkage._combine.AndJoinConditionsLinkage at 0x128866bd0>,
 KeyLinkage(
     keys=(city, _.street[slice(None, 5, None)], unit)
     nleft=315_659
     nright=992_329
     nlinks=3_417
 ))

In [6]:
linkage = address_linker.linkage(clean, dirty)
linkage

KeyLinkage(
    keys=(city, postcode, street, unit)
    nleft=315_659
    nright=992_329
    nlinks=102
)

In [7]:
is_laurel = _.street.contains("4228 LAUREL")
dirty.filter(is_laurel), clean.filter(is_laurel)

(┏━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┓
 ┃ record_id ┃ state  ┃ city      ┃ postcode ┃ street         ┃ unit   ┃
 ┡━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━┩
 │ int64     │ string │ string    │ string   │ string         │ string │
 ├───────────┼────────┼───────────┼──────────┼────────────────┼────────┤
 │    354101 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354102 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354103 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354104 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354105 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354106 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354107 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354108 │ AK     │ ANCHORAGE │ 99508    │ 4228 LAUREL ST │ NULL   │
 │    354109 │ AK     │ ANCHORAGE │ 99508    │ 4228

In [8]:
def value_trigrams(s: ibis.ir.StringValue) -> ibis.ir.ArrayColumn:
    return (
        mismo.text.tokenize(s).map(lambda token: mismo.text.ngrams(token, 3)).flatten()
    )


def trigrams(t: ibis.Table) -> ibis.ir.ArrayColumn:
    return (
        ibis.array(
            [
                value_trigrams(t.city),
                value_trigrams(t.postcode),
                value_trigrams(t.street),
            ]
        )
        .flatten()
        .unique()
        .name("trigrams")
    )


clean = clean.mutate(trigrams(clean)).cache()
dirty = dirty.mutate(trigrams(dirty)).cache()
clean

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┏━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ state  ┃ city         ┃ postcode ┃ street                 ┃ unit   ┃ lon         ┃ lat       ┃ trigrams                ┃
┡━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64     │ string │ string       │ string   │ string                 │ string │ float64     │ float64   │ array<string>           │
├───────────┼────────┼──────────────┼──────────┼────────────────────────┼────────┼─────────────┼───────────┼─────────────────────────┤
│         0 │ AK     │ ANCHOR POINT │ 99556    │ 23837 NIKOLAI STREET   │ NULL   │ -151.746357 │ 59.926617 │ ['STR', 'NIK', ... +20] │
│         1 │ AK     │ ANCHOR POINT │ 99556    │ 24338 MISTY LANE       │ NULL   │ -151.739071 │ 59.919594 │ ['ANE', 'LAN', ... +16] │
│         2 │ AK     │ ANCHOR POINT │ 99556    │ 24715 STERLING HIGHWAY │ NULL   │ -151.755646 │ 59.913984 │ ['WAY', 'GHW', ... +22] │
│         3 │ AK     │ ANCHOR POINT │ 99556    │ 24803 INLET COVE LANE  │ NULL   │ -151.757118 │ 59.912571 │ ['ANE', 'LAN', ... +18] │
│         4 │ AK     │ ANCHOR POINT │ 99556    │ 24825 INLET COVE LANE  │ NULL   │ -151.757657 │ 59.912205 │ ['ANE', 'LAN', ... +18] │
│         5 │ AK     │ ANCHOR POINT │ 99556    │ 24845 INLET COVE LANE  │ NULL   │ -151.757918 │ 59.911979 │ ['ANE', 'LAN', ... +18] │
│         6 │ AK     │ ANCHOR POINT │ 99556    │ 24865 INLET COVE LANE  │ NULL   │ -151.758861 │ 59.911740 │ ['ANE', 'LAN', ... +18] │
│         7 │ AK     │ ANCHOR POINT │ 99556    │ 25100 PEPPER ROAD      │ NULL   │ -151.709791 │ 59.908129 │ ['OAD', 'ROA', ... +17] │
│         8 │ AK     │ ANCHOR POINT │ 99556    │ 25238 STINKBUG STREET  │ NULL   │ -151.692157 │ 59.906734 │ ['BUG', 'TIN', ... +21] │
│         9 │ AK     │ ANCHOR POINT │ 99556    │ 25405 STINKBUG STREET  │ NULL   │ -151.695885 │ 59.903846 │ ['STR', 'BUG', ... +21] │
│         … │ …      │ …            │ …        │ …                      │ …      │           … │         … │ …                       │
└───────────┴────────┴──────────────┴──────────┴────────────────────────┴────────┴─────────────┴───────────┴─────────────────────────┘

In [35]:
easy_linkage = mismo.KeyLinker("state", "city", "postcode", "street")
easy_linkage.pair_counts(clean, dirty)

┏━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ state  ┃ city      ┃ postcode ┃ street                ┃ n     ┃
┡━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ string │ string    │ string   │ string                │ int64 │
├────────┼───────────┼──────────┼───────────────────────┼───────┤
│ AK     │ ANCHORAGE │ 99503    │ 1003 CHUGACH WAY      │  4107 │
│ AK     │ ANCHORAGE │ 99515    │ 2200 MINERVA WAY      │  3120 │
│ AK     │ FAIRBANKS │ 99701    │ 1516 GILLAM WAY       │  2262 │
│ AK     │ ANCHORAGE │ 99515    │ 9600 MORNINGSIDE LOOP │  1740 │
│ AK     │ ANCHORAGE │ 99515    │ 9540 MORNINGSIDE LOOP │  1480 │
│ AK     │ JUNEAU    │ 99801    │ 2800 POSTAL WAY       │  1012 │
│ AK     │ PALMER    │ 99645    │ 1150 S COLONY WAY     │   897 │
│ AK     │ ANCHORAGE │ 99515    │ 9710 MORNINGSIDE LOOP │   860 │
│ AK     │ ANCHORAGE │ 99518    │ 7422 FOXRIDGE WAY     │   840 │
│ AK     │ ANCHORAGE │ 99515    │ 2100 MINERVA WAY      │   744 │
│ …      │ …         │ …        │ …                     │     … │
└────────┴───────────┴──────────┴───────────────────────┴───────┘

In [16]:
clean.group_by(
    "state",
    "city",
    "postcode",
    "street",
).agg(
    n=_.count(),
    units=_.unit.collect(),
).order_by(_.n.desc())

┏━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ state  ┃ city      ┃ postcode ┃ street                    ┃ n     ┃ units                               ┃
┡━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string │ string    │ string   │ string                    │ int64 │ array<string>                       │
├────────┼───────────┼──────────┼───────────────────────────┼───────┼─────────────────────────────────────┤
│ AK     │ VALDEZ    │ 99686    │ 300 N HARBOR DRIVE        │   616 │ ['PIER A', 'PIER B', ... +613]      │
│ AK     │ ANCHORAGE │ 99515    │ 1200 W DIMOND BOULEVARD   │   523 │ ['MICRO', 'TRLR 1001', ... +520]    │
│ AK     │ ANCHORAGE │ 99504    │ 2221 MULDOON ROAD         │   473 │ ['TRLR 1', 'TRLR 10', ... +470]     │
│ AK     │ ANCHORAGE │ 99501    │ 905 RICHARDSON VISTA ROAD │   419 │ ['APT', 'APT 1', ... +417]          │
│ AK     │ ANCHORAGE │ 99508    │ 3240 PENLAND PARKWAY      │   391 │ ['TRLR 100', 'TRLR 101', ... +388]  │
│ AK     │ ANCHORAGE │ 99504    │ 5901 E 6TH AVENUE         │   359 │ ['TRLR 1', 'TRLR 10', ... +356]     │
│ AK     │ ANCHORAGE │ 99507    │ 9499 BRAYTON DRIVE        │   314 │ ['79', 'TRLR 1', ... +311]          │
│ AK     │ JUNEAU    │ 99801    │ 6590 GLACIER HIGHWAY      │   301 │ ['1', '10', ... +298]               │
│ AK     │ ANCHORAGE │ 99504    │ 1001 BONIFACE PARKWAY     │   221 │ ['TRLR 10E1', 'TRLR 10G', ... +218] │
│ AK     │ VALDEZ    │ 99686    │ 315 ATIGUN DRIVE          │   218 │ ['SPACE 1', 'SPACE 10', ... +215]   │
│ …      │ …         │ …        │ …                         │     … │ …                                   │
└────────┴───────────┴──────────┴───────────────────────────┴───────┴─────────────────────────────────────┘

In [10]:
def trigram_to_record_ids(t: ibis.Table) -> ibis.Table:
    counts_table = (
        t.select("record_id", t.trigrams.unnest().name("trigram"))
        .group_by("trigram")
        .agg(
            n_records=_.count(),
            record_ids=_.record_id.collect(),
        )
    )
    counts_table = counts_table.cache()
    return counts_table


trigrams_clean = trigram_to_record_ids(clean)
trigrams_dirty = trigram_to_record_ids(dirty)
trigrams_clean

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ trigram ┃ n_records ┃ record_ids                   ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string  │ int64     │ array<int64>                 │
├─────────┼───────────┼──────────────────────────────┤
│ STY     │       447 │ [122880, 245970, ... +445]   │
│ DIN     │      1269 │ [122880, 122881, ... +1267]  │
│ 572     │       703 │ [122880, 122881, ... +701]   │
│ 741     │       697 │ [122885, 247288, ... +695]   │
│ DRI     │     70660 │ [122891, 122892, ... +70658] │
│ 357     │       573 │ [122901, 123000, ... +571]   │
│ 662     │       421 │ [122912, 246592, ... +419]   │
│ NUG     │       193 │ [122912, 122914, ... +191]   │
│ 380     │      1263 │ [122916, 122970, ... +1261]  │
│ 391     │       735 │ [122919, 122972, ... +733]   │
│ …       │         … │ …                            │
└─────────┴───────────┴──────────────────────────────┘

In [19]:
def rare_trigrams(t: ibis.Table, threshold: int) -> ibis.Table:
    return t.filter(_.n_records < threshold)


trigrams_clean_rare = rare_trigrams(trigrams_clean, 100)
trigrams_dirty_rare = rare_trigrams(trigrams_dirty, 100)
trigrams_clean_rare

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ trigram ┃ n_records ┃ record_ids                ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string  │ int64     │ array<int64>              │
├─────────┼───────────┼───────────────────────────┤
│ JUS     │        64 │ [122968, 122970, ... +62] │
│ HSI     │         6 │ [247366, 247439, ... +4]  │
│ AEL     │        92 │ [247898, 247995, ... +90] │
│ HAE     │        73 │ [247898, 247995, ... +71] │
│ BUF     │        79 │ [248017, 248043, ... +77] │
│ RR-     │         2 │ [250529, 250530]          │
│ HEU     │         4 │ [250747, 250748, ... +2]  │
│ BUD     │        91 │ [250883, 268506, ... +89] │
│ NOX     │        54 │ [250894, 158607, ... +52] │
│ EYV     │        11 │ [251334, 251336, ... +9]  │
│ …       │         … │ …                         │
└─────────┴───────────┴───────────────────────────┘

In [20]:
join_table = ibis.join(
    trigrams_clean_rare,
    trigrams_dirty_rare,
    how="inner",
    predicates="trigram",
    lname="{name}_l",
    rname="{name}_r",
)
join_table

┏━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ trigram ┃ n_records_l ┃ record_ids_l              ┃ n_records_r ┃ record_ids_r              ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string  │ int64       │ array<int64>              │ int64       │ array<int64>              │
├─────────┼─────────────┼───────────────────────────┼─────────────┼───────────────────────────┤
│ TYS     │          22 │ [122880, 296764, ... +20] │          54 │ [617538, 617539, ... +52] │
│ 09T     │           1 │ [210386]                  │          18 │ [990886, 969410, ... +16] │
│ 01S     │          10 │ [165789, 165790, ... +8]  │          30 │ [990935, 990936, ... +28] │
│ ONL     │          46 │ [196105, 229980, ... +44] │          15 │ [991126, 991127, ... +13] │
│ ILR     │          52 │ [297942, 299406, ... +50] │          34 │ [991377, 929079, ... +32] │
│ STS     │          23 │ [248491, 248536, ... +21] │          41 │ [991672, 991673, ... +39] │
│ HTI     │          30 │ [291302, 291323, ... +28] │          37 │ [991783, 991784, ... +35] │
│ EMB     │          55 │ [137194, 138812, ... +53] │          91 │ [992018, 992019, ... +89] │
│ YAW     │           2 │ [255632, 258411]          │           1 │ [992163]                  │
│ ALU     │          81 │ [261882, 261910, ... +79] │          73 │ [395637, 395638, ... +71] │
│ …       │           … │ …                         │           … │ …                         │
└─────────┴─────────────┴───────────────────────────┴─────────────┴───────────────────────────┘

In [26]:
links = join_table.select(
    "trigram",
    record_id_l=_.record_ids_l.unnest(),
    record_id_r=_.record_ids_r.unnest(),
)
print(links.count())
links

┌───────┐
│ 56920 │
└───────┘


┏━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ trigram ┃ record_id_l ┃ record_id_r ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string  │ int64       │ int64       │
├─────────┼─────────────┼─────────────┤
│ TYS     │      122880 │      617538 │
│ TYS     │      296764 │      617539 │
│ TYS     │      203272 │      617540 │
│ TYS     │      203290 │      617541 │
│ TYS     │      203292 │      617542 │
│ TYS     │      203312 │      617543 │
│ TYS     │      203320 │      617544 │
│ TYS     │      203322 │      617545 │
│ TYS     │      203324 │      617546 │
│ TYS     │      122863 │      617547 │
│ …       │           … │           … │
└─────────┴─────────────┴─────────────┘

In [97]:
trigram_linkage = mismo.linkage.LinkTableLinkage(clean, dirty, links)
trigram_linkage = trigram_linkage.cache()
trigram_linkage

LinkTableLinkage(
    left=315_659,
    right=992_329,
    links=56_920,
)

In [98]:
trigram_linkage.links.with_left().with_right()

TypeError: LinksTable._swap_perspective() takes 1 positional argument but 2 were given

In [91]:
clean.count(), trigram_linkage.left.count(), trigram_linkage.left.with_n_links().count()

(┌────────┐
 │ 315659 │
 └────────┘,
 ┌────────┐
 │ 315659 │
 └────────┘,
 ┌────────┐
 │ 315659 │
 └────────┘)

In [93]:
trigram_linkage.left.with_n_links()

┏━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ record_id ┃ state  ┃ city         ┃ postcode ┃ street                 ┃ unit   ┃ lon         ┃ lat       ┃ trigrams                ┃ n_links ┃
┡━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ int64     │ string │ string       │ string   │ string                 │ string │ float64     │ float64   │ array<string>           │ int64   │
├───────────┼────────┼──────────────┼──────────┼────────────────────────┼────────┼─────────────┼───────────┼─────────────────────────┼─────────┤
│         0 │ AK     │ ANCHOR POINT │ 99556    │ 23837 NIKOLAI STREET   │ NULL   │ -151.746357 │ 59.926617 │ ['STR', 'NIK', ... +20] │       0 │
│         1 │ AK     │ ANCHOR POINT │ 99556    │ 24338 MISTY LANE       │ NULL   │ -151.739071 │ 59.919594 │ ['ANE', 'LAN', ... +16] │       0 │
│         2 │ AK     │ ANCHOR POINT │ 99556    │ 24715 STERLING HIGHWAY │ NULL   │ -151.755646 │ 59.913984 │ ['WAY', 'GHW', ... +22] │       0 │
│         3 │ AK     │ ANCHOR POINT │ 99556    │ 24803 INLET COVE LANE  │ NULL   │ -151.757118 │ 59.912571 │ ['ANE', 'LAN', ... +18] │       0 │
│         4 │ AK     │ ANCHOR POINT │ 99556    │ 24825 INLET COVE LANE  │ NULL   │ -151.757657 │ 59.912205 │ ['ANE', 'LAN', ... +18] │       0 │
│         5 │ AK     │ ANCHOR POINT │ 99556    │ 24845 INLET COVE LANE  │ NULL   │ -151.757918 │ 59.911979 │ ['ANE', 'LAN', ... +18] │       0 │
│         6 │ AK     │ ANCHOR POINT │ 99556    │ 24865 INLET COVE LANE  │ NULL   │ -151.758861 │ 59.911740 │ ['ANE', 'LAN', ... +18] │       0 │
│         7 │ AK     │ ANCHOR POINT │ 99556    │ 25100 PEPPER ROAD      │ NULL   │ -151.709791 │ 59.908129 │ ['OAD', 'ROA', ... +17] │       0 │
│         8 │ AK     │ ANCHOR POINT │ 99556    │ 25238 STINKBUG STREET  │ NULL   │ -151.692157 │ 59.906734 │ ['BUG', 'TIN', ... +21] │       0 │
│         9 │ AK     │ ANCHOR POINT │ 99556    │ 25405 STINKBUG STREET  │ NULL   │ -151.695885 │ 59.903846 │ ['STR', 'BUG', ... +21] │       0 │
│         … │ …      │ …            │ …        │ …                      │ …      │           … │         … │ …                       │       … │
└───────────┴────────┴──────────────┴──────────┴────────────────────────┴────────┴─────────────┴───────────┴─────────────────────────┴─────────┘

In [94]:
trigram_linkage.left.with_n_links().n_links.topk(10)

┏━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ n_links ┃ n_links_count ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ int64   │ int64         │
├─────────┼───────────────┤
│       0 │        295003 │
│       1 │         17179 │
│       2 │          3158 │
│       3 │           279 │
│       4 │            39 │
│       5 │             1 │
└─────────┴───────────────┘

In [89]:
trigram_linkage.left.count()

┌────────┐
│ 315659 │
└────────┘

In [96]:
trigram_linkage.left.link_counts()

28164


┏━━━━━━━━━┳━━━━━━━━━━━┓
┃ n_links ┃ n_records ┃
┡━━━━━━━━━╇━━━━━━━━━━━┩
│ int64   │ int64     │
├─────────┼───────────┤
│       3 │       279 │
│       1 │     17179 │
│       0 │      7507 │
│       4 │        39 │
│   16486 │         0 │
│       5 │         1 │
│       2 │      3158 │
└─────────┴───────────┘

In [50]:
links_full = trigram_linkage.links.with_left().with_right()
links_full = links_full.select(sorted(links_full.columns))
links_full


┏━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ city_l    ┃ city_r    ┃ lat_l     ┃ lon_l       ┃ postcode_l ┃ postcode_r ┃ record_id_l ┃ record_id_r ┃ state_l ┃ state_r ┃ street_l               ┃ street_r            ┃ trigram ┃ trigrams_l              ┃ trigrams_r              ┃ unit_l ┃ unit_r ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ string    │ string    │ float64   │ float64     │ string     │ string     │ int64       │ int64       │ string  │ string  │ string                 │ string              │ string  │ array<string>           │ array<string>           │ string │ string │
├───────────┼───────────┼───────────┼─────────────┼────────────┼────────────┼─────────────┼─────────────┼─────────┼─────────┼────────────────────────┼─────────────────────┼─────────┼─────────────────────────┼─────────────────────────┼────────┼────────┤
│ ANCHORAGE │ ANCHORAGE │ 61.187445 │ -149.724311 │ 99504      │ 99504      │       49141 │      247399 │ AK      │ AK      │ 8400 RAGGED TOP CIRCLE │ 8321 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187609 │ -149.724384 │ 99504      │ 99504      │       49155 │      247400 │ AK      │ AK      │ 8401 RAGGED TOP CIRCLE │ 8321 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187424 │ -149.724106 │ 99504      │ 99504      │       49185 │      247401 │ AK      │ AK      │ 8410 RAGGED TOP CIRCLE │ 8321 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187590 │ -149.724190 │ 99504      │ 99504      │       49196 │      247402 │ AK      │ AK      │ 8411 RAGGED TOP CIRCLE │ 8321 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187404 │ -149.723910 │ 99504      │ 99504      │       49227 │      247485 │ AK      │ AK      │ 8420 RAGGED TOP CIRCLE │ 8340 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187630 │ -149.723971 │ 99504      │ 99504      │       49244 │      247486 │ AK      │ AK      │ 8421 RAGGED TOP CIRCLE │ 8340 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187387 │ -149.723743 │ 99504      │ 99504      │       49279 │      247487 │ AK      │ AK      │ 8430 RAGGED TOP CIRCLE │ 8340 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187686 │ -149.723754 │ 99504      │ 99504      │       49291 │      247532 │ AK      │ AK      │ 8431 RAGGED TOP CIRCLE │ 8351 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187371 │ -149.723576 │ 99504      │ 99504      │       49308 │      247533 │ AK      │ AK      │ 8440 RAGGED TOP CIRCLE │ 8360 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ ANCHORAGE │ ANCHORAGE │ 61.187706 │ -149.723582 │ 99504      │ 99504      │       49316 │      247534 │ AK      │ AK      │ 8441 RAGGED TOP CIRCLE │ 8360 RAGGED TOP CIR │ AGG     │ ['IRC', 'CIR', ... +18] │ ['CIR', 'GGE', ... +15] │ NULL   │ NULL   │
│ …         │ …         │         … │           … │ …          │ …          │           … │           … │ …       │ …       │ …                      │ …                   │ …       │ …                     

In [46]:
links_full.mutate(jaccard=mismo.sets.jaccard(_.trigrams_l, _.trigrams_r)).order_by(
    _.jaccard.desc()
).filter(
    _.jaccard < 1,
    _.jaccard > 0.5,
)

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓
┃ city_l      ┃ city_r      ┃ lat_l     ┃ lon_l       ┃ postcode_l ┃ postcode_r ┃ record_id_l ┃ record_id_l_right ┃ record_id_r ┃ record_id_r_right ┃ state_l ┃ state_r ┃ street_l             ┃ street_r          ┃ trigram ┃ trigrams_l              ┃ trigrams_r              ┃ unit_l ┃ unit_r ┃ jaccard  ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩
│ string      │ string      │ float64   │ float64     │ string     │ string     │ int64       │ int64             │ int64       │ int64             │ string  │ string  │ string               │ string            │ string  │ array<string>           │ array<string>           │ string │ string │ float64  │
├─────────────┼─────────────┼───────────┼─────────────┼────────────┼────────────┼─────────────┼───────────────────┼─────────────┼───────────────────┼─────────┼─────────┼──────────────────────┼───────────────────┼─────────┼─────────────────────────┼─────────────────────────┼────────┼────────┼──────────┤
│ EAGLE RIVER │ EAGLE RIVER │ 61.277189 │ -149.371916 │ 99577      │ 99577      │      135092 │            135092 │      600173 │            600173 │ AK      │ AK      │ 6270 MAGNAVIEW DRIVE │ 6270 MAGNAVIEW DR │ GNA     │ ['VIE', 'IEW', ... +17] │ ['VIE', 'IEW', ... +16] │ NULL   │ NULL   │ 0.947368 │
│ ANCHORAGE   │ ANCHORAGE   │ 61.119973 │ -149.889473 │ 99515      │ 99515      │       83053 │             83053 │      406614 │            406614 │ AK      │ AK      │ 11150 SUKDU WAY      │ 11150 SUKDU       │ SUK     │ ['WAY', 'KDU', ... +15] │ ['KDU', 'UKD', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ EAGLE RIVER │ EAGLE RIVER │ 61.258928 │ -149.466730 │ 99577      │ 99577      │      134972 │            134972 │      599755 │            599755 │ AK      │ AK      │ 4241 ELKHORN DRIVE   │ 4241 ELKHORN DR   │ KHO     │ ['ORN', 'LKH', ... +15] │ ['ORN', 'LKH', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ ANCHORAGE   │ ANCHORAGE   │ 61.119973 │ -149.889473 │ 99515      │ 99515      │       83053 │             83053 │      406614 │            406614 │ AK      │ AK      │ 11150 SUKDU WAY      │ 11150 SUKDU       │ KDU     │ ['WAY', 'KDU', ... +15] │ ['KDU', 'UKD', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ EAGLE RIVER │ EAGLE RIVER │ 61.309331 │ -149.552227 │ 99577      │ 99577      │      135979 │            135979 │      601878 │            601878 │ AK      │ AK      │ 9823 DINAAKA DRIVE   │ 9823 DINAAKA DR   │ AAK     │ ['NAA', 'AAK', ... +15] │ ['NAA', 'AAK', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ EAGLE RIVER │ EAGLE RIVER │ 61.308055 │ -149.553310 │ 99577      │ 99577      │      135913 │            135913 │      601731 │            601731 │ AK      │ AK      │ 9706 DINAAKA DRIVE   │ 9706 DINAAKA DR   │ AAK     │ ['NAA', 'AKA', ... +15] │ ['NAA', 'AKA', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ EAGLE RIVER │ EAGLE RIVER │ 61.258928 │ -149.466730 │ 99577      │ 99577      │      134972 │            134972 │      599763 │            599763 │ AK      │ AK      │ 4241 ELKHORN DRIVE   │ 4241 ELKHORN DR   │ LKH     │ ['ORN', 'LKH', ... +15] │ ['ORN', 'LKH', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ ANCHORAGE   │ ANCHORAGE   │ 61.119973 │ -149.889473 │ 99515      │ 99515      │       83053 │             83053 │      406614 │            406614 │ AK      │ AK      │ 11150 SUKDU WAY      │ 11150 SUKDU       │ UKD     │ ['WAY', 'KDU', ... +15] │ ['KDU', 'UKD', ... +14] │ NULL   │ NULL   │ 0.941176 │
│ EAGLE RIVER │ EAGLE RIVER │ 61.254310 │ -149.2

In [43]:
linkage.link_counts_chart()

alt.HConcatChart(...)

In [8]:
linkage.links.count()

┌────────┐
│ 429374 │
└────────┘

In [ ]:
mismo.linkage.union.implementations

[(<function mismo.block._key_linkage_combined._all_are_key_linkages(*linkages: 'BaseLinkage') -> 'bool'>,
  <function mismo.block._key_linkage_combined.register_combiners.<locals>.<lambda>(*linkages)>),
 (<function mismo.linkage._combine.Combiner.__init__.<locals>.<lambda>(*x)>,
  <function mismo.linkage._combine.Union.default(first: 'BaseLinkage', *linkages: 'BaseLinkage') -> 'BaseLinkage'>)]

In [11]:
mismo.linkage.union(linkage, linkage)

Let's clean this up a bit:
- clean up whitespace
- convert the `classes` column to actual arrays (they really represent sets).
  Each element in `classes` is a 4-character IPC technical code that is like a tag
  for the patent. Similar patents will have similar tags.
- convert `coauthors` into a sorted list of tokens.

Of course, this sort of cleaning only happens after a lot of exploring the
data manually. This isn't anything in particular to record linkage,
but this needs to happen for any sort of data analysis.

If you're not familiar with Ibis, this is useful to give you a sense
of what it's API is like. Mismo tries to just use Ibis's API whenever possible,
and doesn't re-invent the wheel.


In [ ]:
from mismo.text import norm_whitespace


def clean_names(names: ir.StringValue) -> ir.StringValue:
    names = norm_whitespace(names)
    names = names.upper()
    # Only want to keep letters, numbers, and spaces
    names = names.re_replace("[^0-9A-Z ]", "")
    # Some company names have "NV" or "BV" in them, which is a Dutch
    # classification of businesses sort of like LLC vs Corporation in the US.
    # These seem to just be noise and don't help us distinguish between
    # different companies, so we want to remove them.
    # However, sometimes the names are ill-formed or have other junk in them
    # like "philipsN V" or "philipsN.V."
    names = names.re_replace(r"B\s*V", "")
    names = names.re_replace(r"N\s*V", "")
    # Also remove other common stopwords that usually just add noise
    names = names.re_replace(r"\bCO\b", "")
    names = names.re_replace(r"\bCORP\b", "")
    names = names.re_replace(r"\bLTD\b", "")
    names = names.re_replace(r"\bINC\b", "")
    names = names.re_replace(r"\bLLC\b", "")
    names = names.re_replace(r"\bNL\b", "")
    # Now have to do whitespace fixup again
    names = norm_whitespace(names)
    return names


def clean_coauthors(s: ir.StringValue) -> ir.ArrayValue:
    """Turn a 'list' of coauthors into a sorted array of cleaned name tokens.

    The raw string looks like:
    "DAVID E  SNYDER**THOMAS D  LYSTER"
    With "**" as the separator. However, entire names are not always useful
    to differentiate between people: there are some records where a coauthor is
    listed as "ALKEMA DUURT PIETER WILLEM" and another as "ALKEMA DUURT".
    These are probably the same person, but if we compared entire names with each other
    that would be a false negative. So instead we turn this into a sorted array of
    name tokens, eg for the example above:
    ["DAVID", "E", "SNYDER", "THOMAS", "D", "LYSTER"].

    Except we also want to remove initials and other stopwords like "VAN"
    so scrub all tokens less than 3 characters long.
    """
    s = s.upper().nullif("NONE").replace("**", " ")
    s = norm_whitespace(s)
    return s.split(" ").unique().sort().filter(lambda x: x.length() >= 3)


def clean_classes(s: ir.StringValue) -> ir.ArrayValue:
    return s.upper().split("**").map(norm_whitespace).sort()


cleaned_names = clean_names(_.name)

featured = patents.select(
    "record_id",
    "label_true",
    "name_true",
    "name",
    name_cleaned=cleaned_names,
    name_tokens=cleaned_names.split(" ").map(norm_whitespace).sort(),
    latitude=_.latitude.nullif(0),
    longitude=_.longitude.nullif(0),
    coauthors=clean_coauthors(patents.coauthors),
    classes=clean_classes(patents.classes),
)
featured

OK, now it's time to block! This is where we generate comparisons between records.
If we were naive and generated all possible comparisons from N record,
you would end up with N^2 comparisons. For our small dataset of ~2000 records
we would be able to get away with this, but for datasets much larger than this
it would be infeasible.

For blocking, we specify an collection of `Blocker`s. A Blocker is just
a callable of the form `Table, Table, **kwargs -> Table` that takes in two tables
and returns a table of pairs of records for further processing.

In [ ]:
name_blocker = mismo.block.ConditionBlocker(
    lambda left, right, **_kwargs: left.name_cleaned[0:3] == right.name_cleaned[0:3],
    name="Name First 3",
)
name_blocker(featured, featured)

The result of that was the two tables joined together, with a `_l` added
to all the columns from the left table, and a `_r` added to all the columns
from the right table.

By using this simple API, Blockers are easily composable and extendable.
For example, we can make an ensemble Blocker that takes multiple
other blockers, and returns the union of all their pairs.

In [ ]:
sub_blockers = [
    name_blocker,
    # Often, you want to look at pairs where records share some key.
    # There is KeyBlocker for this. This is equivalent to
    # `lambda left, right: left.coauthors == right.coauthors`
    mismo.block.KeyBlocker("coauthors", name="Coauthors Exact"),
    # You can also pass in deferreds for more complex logic.
    # And notice that the .unnest() actually changes the cardinality of the
    # key column, but the KeyBlocker can handle this.
    # Here, we are finding pairs where the arrays of classes have at least
    # one in common.
    mismo.block.KeyBlocker(_.classes.unnest(), name="Any Class"),
    # CoordinateBlocker is merely a class that implements a __call__ method,
    # which makes it a callable similar to the lambda above.
    # You can implement your own blockers by implementing __call__ for
    # any class. You could do complex logic in there, like
    # - Dropping any null records
    # - Using a UDF to computing the doublemetaphone of the name on the fly.
    # - Using a UDF to do an approximate-nearest-neighbor search in a vector database
    # - whatever you want! As long as it follows the required signature.
    mismo.lib.geo.CoordinateBlocker(
        distance_km=10, name="Coordinates Close", lat="latitude", lon="longitude"
    ),
]
# If you didn't care about the names of the rules, you could also just pass
# in the blockers directly, eg `rules=["classes", _.name_cleaned[0:3], ...]`.

featured = featured.cache()
# by passing in `labels=True`, an additional column `blocking_rules` is added
# to the output that shows which rules were used to generate each pair.
# This is slower, but later we want to see how many pairs each rule generated.
blocker = mismo.block.UnionBlocker(*sub_blockers, labels=True)
blocked = blocker(featured, featured)
blocked = blocked.cache()
blocked

In [ ]:
blocked.blocking_rules.unnest().value_counts()

By blocking, we reduced the number of needed pairs by a large factor.
In larger datasets, and with better blocking rules, this would be even more!

In [ ]:
n_comparisons = blocked.count().execute()
n_naive = mismo.block.n_naive_comparisons(featured)
reduction_ratio = n_comparisons / n_naive
n_naive, n_comparisons, reduction_ratio

We can also inspect which blocking rules were most to blame for the generated
pairs. If some rules generate a huge amount of comparisons, it might be worth
trying to make them more restrictive so we get better performance. Or, if some
blocking rules aren't generating any comparisons, that might be an indication
that we have a bug in there somewhere.

In [ ]:
mismo.block.upset_chart(blocked)

OK, now that we have our candidate pairs generated, let's actually do the
comparing of pairs.

In Mismo, similar to blocking, we use a single `PComparer` protocol to do all
comparisons. A `PComparer` is simply a callable of the form `Table, **kwargs -> Table`
that takes a table of blocked record pairs and returns a modified version.
Typical things a `PComparer` might do include

- Dropping some comparisons that are definitely not matches.
- Adding a score column for how much some feature matches, eg how well
  the address matches.
- Combining these individual per-feature scores into an overall score for the pair.

Here, we use one of the most common methods using LevelComparers,
each of which represents a measurement of similarity based on some dimension
(eg "location", "name", "price", etc).
Each LevelComparer is composed of MatchLevels, which represent discrete levels of agreement
(eg "exactly", "within 100km", "one or both values null", etc)

In [ ]:
from mismo.compare import LevelComparer, MatchLevel
from mismo.lib.geo import distance_km
from mismo.sets import jaccard


class NameMatchLevel(MatchLevel):
    EXACT = 0
    SHARE_1_TOKEN = 1
    SHARE_2_PLUS_TOKENS = 2
    ELSE = 3


name_comparison = LevelComparer(
    name="Name",
    levels=NameMatchLevel,
    cases=[
        (_.name_cleaned_l == _.name_cleaned_r, NameMatchLevel.EXACT),
        (
            _.name_tokens_l.intersect(_.name_tokens_r).length() == 1,
            NameMatchLevel.SHARE_1_TOKEN,
        ),
        (
            _.name_tokens_l.intersect(_.name_tokens_r).length() >= 2,
            NameMatchLevel.SHARE_2_PLUS_TOKENS,
        ),
        (True, NameMatchLevel.ELSE),
    ],
)


class ClassMatchLevel(MatchLevel):
    EXACT = 0
    SHARE_1_CLASS = 1
    SHARE_2_PLUS_CLASSES = 2
    ELSE = 3


classes_comparison = LevelComparer(
    name="Classes",
    levels=ClassMatchLevel,
    cases=[
        (_.classes_l == _.classes_r, ClassMatchLevel.EXACT),
        (
            _.classes_l.intersect(_.classes_r).length() == 1,
            ClassMatchLevel.SHARE_1_CLASS,
        ),
        (
            _.classes_l.intersect(_.classes_r).length() >= 2,
            ClassMatchLevel.SHARE_2_PLUS_CLASSES,
        ),
        (True, ClassMatchLevel.ELSE),
    ],
)


class CoauthorMatchLevel(MatchLevel):
    JACCARD_50 = 0
    JACCARD_25 = 1
    JACCARD_10 = 2
    JACCARD_02 = 3
    ELSE = 4


coauthors_comparison = LevelComparer(
    name="Coauthors",
    levels=CoauthorMatchLevel,
    cases=[
        (jaccard(_.coauthors_l, _.coauthors_r) >= 0.5, CoauthorMatchLevel.JACCARD_50),
        (jaccard(_.coauthors_l, _.coauthors_r) >= 0.25, CoauthorMatchLevel.JACCARD_25),
        (jaccard(_.coauthors_l, _.coauthors_r) >= 0.1, CoauthorMatchLevel.JACCARD_10),
        (jaccard(_.coauthors_l, _.coauthors_r) >= 0.02, CoauthorMatchLevel.JACCARD_02),
        (True, CoauthorMatchLevel.ELSE),
    ],
)


class CoordMatchLevel(MatchLevel):
    EXACT = 0
    WITHIN_10KM = 1
    WITHIN_100KM = 2
    BOTH_MISSING = 3
    ONE_MISSING = 4
    ELSE = 5


coords_comparison = LevelComparer(
    name="Coords",
    levels=CoordMatchLevel,
    cases=[
        (_.latitude_l == _.latitude_r, CoordMatchLevel.EXACT),
        (
            distance_km(
                lat1=_.latitude_l,
                lon1=_.longitude_l,
                lat2=_.latitude_r,
                lon2=_.longitude_r,
            )
            <= 10,
            CoordMatchLevel.WITHIN_10KM,
        ),
        (
            distance_km(
                lat1=_.latitude_l,
                lon1=_.longitude_l,
                lat2=_.latitude_r,
                lon2=_.longitude_r,
            )
            <= 100,
            CoordMatchLevel.WITHIN_100KM,
        ),
        (
            ibis.and_(
                _.latitude_l.isnull() | _.longitude_l.isnull(),
                _.latitude_r.isnull() | _.longitude_r.isnull(),
            ),
            CoordMatchLevel.BOTH_MISSING,
        ),
        (
            ibis.or_(
                _.latitude_l.isnull() | _.longitude_l.isnull(),
                _.latitude_r.isnull() | _.longitude_r.isnull(),
            ),
            CoordMatchLevel.ONE_MISSING,
        ),
        (True, CoordMatchLevel.ELSE),
    ],
)

comparers = [
    name_comparison,
    classes_comparison,
    coauthors_comparison,
    coords_comparison,
]

compared = blocked
for comp in comparers:
    compared = comp(compared)
compared = compared.cache()
compared

The result above is the blocked table, with a column added for every `Comparer`.
The value of each column is the level that the record pair matched at.
For example, there is now a "Name" column,
filled with values like "exact", "share 1 token", etc.

Now that we have our features, we can use the Fellegi-Sunter model to train weights
for each of these features. This is a probabilistic model that is based on the concept
of odds. When you see an exact match on name, that increases the odds of a match
by some amount, maybe 10x. When you see a non-match on name, that decreases the odds
of a match by some amount, maybe 0.1x. We can either train this from labeled data,
or we can use unlabeled data using an algorithm called "Expectation Maximization".

In [ ]:
weights = mismo.fs.train_using_em(comparers, featured, featured, max_pairs=100_000)
# Can save and load weights
# weights.to_json("weights.json")
# weights = ComparisonWeights.from_json("weights.json")
weights.plot()

In the above plot, you can see the odds for "share 1 class" and "share 2+ classes"
are nearly the same. That doesn't quite make sense, and might be an indicator
that we should look more into this to see if there is a problem with the features
that we are using. But for now let's just move on.

Use the weights to score the record pairs, finding the odds for each
`Comparer`, and then combining them into an overall odds for the record pair.

In [ ]:
scored = weights.score_compared(compared)
scored = scored.cache()
scored

We can plot these compared pairs.
We can see which comparison levels are most common,
which occur together,
which lead to matches, and which lead to non-matches.

The exact match levels have the highest odds, and the
else levels have the lowest. The other levels are somewhere in between.

In [ ]:
mismo.compare.compared_dashboard(compared, comparers=comparers, weights=weights)

It looks like an odds of 50 seems to separate the pairs between non-matches
and matches.
If I hover over the above chart, I can see that pretty much all the "else" comparisons
are in the low cluster, and all the "exact" comparisons are in the high cluster.

In [ ]:
odds_threshold = 50
(scored.odds >= odds_threshold).value_counts()

Let's be really picky and only take the most likely matches as true matches, and
then perform connected components to label each patent with its inventor:

In [ ]:
links = scored[_.odds >= odds_threshold]
links = links.cache()
print(links.count().execute())
labeled = mismo.cluster.connected_components(links=links, records=featured)
labeled = labeled.relocate("component", after="label_true").order_by("component")
print(labeled.count().execute())
labeled

Now let's evaluate how good our labeling is. Mismo wraps all of the evaluation
metrics from sklearn, so we can use them with Ibis Tables.

In [ ]:
labels_true = labeled.select("record_id", label=_.label_true)
labels_pred = labeled.select("record_id", label=_.component)
print(mismo.cluster.adjusted_rand_score(labels_true, labels_pred))
print(mismo.cluster.homogeneity_score(labels_true, labels_pred))
print(mismo.cluster.completeness_score(labels_true, labels_pred))